In [ ]:
import pdfplumber
import pandas as pd
import re
from tqdm import tqdm


class DataStore:
        def __init__(self, data=[]):
            self.parsed_data = data

        def __len__(self):
            return len(self.parsed_data)
        
        def __getitem__(self, ix):
            return self.parsed_data[ix]
        
        def append(self, data):
            data = {
                'heading': data[0][0],
                'unit': data[0][1],
                'table': data[1]
            }
            self.parsed_data.append(data)

class PDFParser:        
    def __init__(self, file_path):
        self.file_path = file_path

    def get_text(self, page):
        page_text = page.extract_text()
        try:
            heading = re.search(r'FINAL ESTIMATE OF(.*?)\n', page_text, re.DOTALL).group(1).strip() if page_text else ''
            unit_search = re.search(r'\((Avg\..*|Production.*|Area.*)\)', page_text, re.DOTALL) if page_text else ''
            unit = unit_search.group(1).strip() if unit_search else ''
        except AttributeError as e:
            return None
        return None if heading == '' else (heading, unit)

    def get_tables(self, page):
        settings = {
            "vertical_strategy": "text",
            "horizontal_strategy": 'text'
        }   
        table = page.extract_table(table_settings=settings)
        table_df = pd.DataFrame(table)
        header = page.extract_table()
        header_df = pd.DataFrame(header).iloc[:2]
        table_df.at[0] = header_df.iloc[0]
        table_df.at[1] = header_df.iloc[1]
        return table_df.iloc[:-1]

    def parse_pdf(self):
        data = DataStore()
        print('Reading Pages: ')
        with pdfplumber.open(self.file_path) as pdf:
            for page in tqdm(pdf.pages):
                type_metric_col = self.get_text(page)
                if type_metric_col is None:
                    continue
                table = self.get_tables(page)
                data.append((type_metric_col, table))
            print('Pages Read Successfully!')
        return data

In [ ]:
file_path = 'pdf_files/kharif_links_2023-24.pdf'
parser = PDFParser(file_path)
parsed_data = parser.parse_pdf()

Reading Pages: 
Pages Read Successfully!


In [ ]:
import numpy as np
import re

def create_cleaned_table(data, year_col):
    df = data['table'].copy(deep=True)

    # Fix column names
    df.iloc[0].ffill(inplace=True, axis=0)
    if df.iloc[1, 0] is None:
        df.at[0, 0], df.at[1, 0] = df.iloc[0, 0].split(' /\n')
    if data['unit'] == '':
        df.at[0, 0], df.at[1, 0] = df.iloc[1, 0], df.iloc[0, 0]
    df.at[0, 0] = df.iloc[0, 0].upper().strip()
    df.at[1, 0] = df.iloc[1, 0].upper().strip()
    df.columns = pd.MultiIndex.from_arrays([df.iloc[0], df.iloc[1]])
    df = df.drop([0, 1]).reset_index(drop=True)
    df = df.replace('', np.nan)
    df.dropna(axis=0, how='all', inplace=True)
    if data['unit'] == '':
        df.columns = df.columns.swaplevel(0, 1)
        df = df[['DIVISIONS', year_col]]
        df.columns = df.columns.droplevel(1)
    else:
        df = df[['DIVISIONS', year_col]]
        df.columns = df.columns.droplevel(0)
    df = df[~(df['DISTRICTS'].str.upper().str.contains('PUNJAB'))].reset_index(drop=True)

    # Add divisions
    df['DIVISIONS'] = ''
    last_div = ''
    for ix, row in df.iterrows():
        district_val = row['DISTRICTS']
        extract = re.search(r'(.*) D.*V:', district_val, flags = re.IGNORECASE)
        if extract:
            last_div = extract.group(1)
            df.loc[ix, 'DIVISIONS'] = extract.group(1)
            df.drop(index=ix, axis=0, inplace=True)
            continue
        df.loc[ix, 'DIVISIONS'] = last_div
    df['DIVISIONS'] = df['DIVISIONS'].map(lambda x: x.title())

    # Clean Table for storage
    col_list = df.columns.tolist()
    df = pd.melt(df, id_vars=[col_list[0], col_list[-1]], var_name='MEASURE_TYPE', value_name='MEASURE_VALUE')
    df['TITLE'] = data['heading']
    df['YEAR'] = int(year_col.split('-')[0])

    # Add unit value
    if data['unit'] == '':
        df['UNIT'] = ''
        for ix, row in df.iterrows():
            row_val = row['MEASURE_TYPE']
            df.loc[ix, 'MEASURE_TYPE'], df.loc[ix, 'UNIT'] = re.search(r'(.*)\((.*)\)', row_val).groups()

    else:
        df['UNIT'] = data['unit']
    def convert_to_numeric(val):
        try:
            return str(float(val))
        except ValueError:
            return 'NOVAL'
    df['MEASURE_VALUE'] = df['MEASURE_VALUE'].apply(convert_to_numeric)
    df = df[df['MEASURE_VALUE'] != 'NOVAL']
    df['MEASURE_VALUE'] = df['MEASURE_VALUE'].astype(np.float32)
    return df[['YEAR', 'TITLE', 'DIVISIONS', 'DISTRICTS', 'MEASURE_TYPE', 'UNIT', 'MEASURE_VALUE']].reset_index()

In [ ]:
val = parsed_data[62]
df = create_cleaned_table(val, '2023-24')

0: COTTON CROP
1: COTTON CROP
2: COTTON CROP
3: COTTON CROP
4: COTTON CROP
5: RICE CROP
6: RICE (CLEAN) CROP
7: RICE (CLEAN) CROP
8: RICE CROP
9: RICE (CLEAN) CROP
10: MAIZE (AUTUMN) CROP
11: MAIZE (AUTUMN) CROP
12: MAIZE (AUTUMN) CROP
13: MAIZE (AUTUMN) CROP
14: MAIZE (AUTUMN) CROP
15: SUGARCANE CROP
16: SUGARCANE CROP
17: SUGARCANE CROP
18: SUGARCANE CROP
19: SUGARCANE CROP
20: MOONG CROP
21: MOONG CROP
22: MOONG CROP
23: MOONG CROP
24: MOONG CROP
25: MASH CROP
26: MASH CROP
27: MASH CROP
28: MASH CROP
29: MASH CROP
30: OTHER KHARIF PULSES CROPS
31: OTHER KHARIF PULSES CROPS
32: GROUNDNUT CROP
33: GROUNDNUT CROP
34: GROUNDNUT CROP
35: GROUNDNUT CROP
36: GROUNDNUT CROP
37: SESAMUM CROP
38: SESAMUM CROP
39: SESAMUM CROP
40: SESAMUM CROP
41: SESAMUM CROP
42: GUARSEED CROP
43: GUARSEED CROP
44: GUARSEED CROP
45: GUARSEED CROP
46: GUARSEED CROP
47: JOWAR CROP
48: JOWAR CROP
49: JOWAR CROP
50: JOWAR CROP
51: JOWAR CROP
52: BAJRA CROP
53: BAJRA CROP
54: BAJRA CROP
55: BAJRA CROP
56: BAJRA C

In [ ]:
df

    index  YEAR       TITLE  ... MEASURE_TYPE          UNIT MEASURE_VALUE
0       2  2023  TENDA CROP  ...        AREA       Hectares         490.0
1       3  2023  TENDA CROP  ...        AREA       Hectares           NaN
2       4  2023  TENDA CROP  ...        AREA       Hectares           NaN
3       5  2023  TENDA CROP  ...        AREA       Hectares           NaN
4       6  2023  TENDA CROP  ...        AREA       Hectares           NaN
..    ...   ...         ...  ...          ...           ...           ...
77     81  2023  TENDA CROP  ...   AVG.YIELD   Kg / Hectare           NaN
78     82  2023  TENDA CROP  ...   AVG.YIELD   Kg / Hectare        6721.0
79     83  2023  TENDA CROP  ...   AVG.YIELD   Kg / Hectare       21548.0
80     84  2023  TENDA CROP  ...   AVG.YIELD   Kg / Hectare        9928.0
81     85  2023  TENDA CROP  ...   AVG.YIELD   Kg / Hectare        8155.0

[82 rows x 8 columns]
